## Evaluating the Final Model
In this notebook, I evaluate the final model by comparing its performance to the baselines created in the Data Exploration and Visualization notebook.  Below I bring in a the necessary libraries, data and a copy of the model from the previous notebook along with its necessary functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from scipy import stats
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
import pickle
pickle_in=open("cleaned_data.pickle","rb")
df=pickle.load(pickle_in)
# I need to drop two columns that I left in for the visualization notebook
df.drop(['home_score','away_score'],axis=1,inplace=True)
# This is the same function from the previous notebook and it will be used to
# evaluate model performance
def calc_return(X_analyse):
    total_risk=[]
    total_reward=[]
    equal_bet_return=[]
    for i in range(len(X_analyse)):
        k=pd.DataFrame(X_analyse.iloc[i]).transpose()
        k.reset_index(drop=True,inplace=True)
        if int(k.preds[0])==1:
            if int(k.real[0])==1:
                if int(k.home_money[0])<0:
                    risk=k.home_money[0]
                    reward=100
                else:
                    risk=-100
                    reward=k.home_money[0]
            else:
                if k.home_money[0]<0:
                    risk=k.home_money[0]
                    reward=k.home_money[0]
                else:
                    risk=-100
                    reward=-100
        else:
            if int(k.real[0])==0:
                if k.away_money[0]<0:
                    risk=k.away_money[0]
                    reward=100
                else:
                    risk=-100
                    reward=k.away_money[0]
            else:
                if k.away_money[0]<0:
                    risk=k.away_money[0]
                    reward=k.away_money[0]
                else:
                    risk=-100
                    reward=-100
        total_risk.append(risk)
        total_reward.append(reward)
        equal_bet_winnings=reward/-risk*100
        equal_bet_return.append(equal_bet_winnings)
    natural_ror=round(-np.mean(total_reward)/np.mean(total_risk)*100,2)
    equal_bet_ror=round(np.mean(equal_bet_return),2)
    return natural_ror,equal_bet_ror
# This is a function for creating train-test splits that will work with my way of
# scoring model performance based on real world return on risk
def test_split(data,test_size,random_state):
    shuf_df=data.sample(frac=1,random_state=random_state)
    shuf_df.reset_index(drop=True,inplace=True)
    df2=shuf_df.copy()
    # This seperates the dataframe into data and target 
    X_temp=df2[df2.columns[1:]]
    y=df2.home_win
    # This standardized the data
    scaler=StandardScaler()
    X_s = scaler.fit_transform(X_temp)
    X=pd.DataFrame(X_s)
    # This does the train-test split in a way that I can carry through the odds values in order to calculate
    # the real-world usefulness of the model
    if len(X)==len(y):
        split_value=int(round(len(X)*(1-test_size),0))
        X_train=X.iloc[0:split_value]
        X_test=X.iloc[split_value:len(X)]
        y_train=y.iloc[0:split_value]
        y_test=y.iloc[split_value:len(X)]
        home_money=shuf_df.iloc[:,-2]
        away_money=shuf_df.iloc[:,-1]
    return X_train,X_test,y_train,y_test,home_money,away_money,split_value

In [ ]:
results_svm_xg=pd.DataFrame([])
for j in range(0,50):
    X_train,X_test,y_train,y_test,home_money,away_money,split_value=test_split(df,.1,j*3)
    svm_clf=svm.SVC(C=6,kernel='linear',random_state=j*3)
    svm_clf.fit(X_train,y_train)
    svm_pred=svm_clf.predict(X_test)
    boost=xgb.XGBClassifier(learning_rate=.001,max_depth=50,
                            min_child_weight=10,n_estimators=200,subsample=.4,gamma=10,random_state=j*3)
    boost.fit(X_train,y_train)
    boost_pred=boost.predict(X_test)
    bets=pd.DataFrame([])
    for i in range(len(y_test)):
        if svm_pred[i]+boost_pred[i]==2:
            bets=bets.append(pd.DataFrame({'preds':1,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                     'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
        else:
            None
        if svm_pred[i]+boost_pred[i]==0:
            bets=bets.append(pd.DataFrame({'preds':0,'real':y_test[i+split_value],'home_money':home_money[i+split_value],
                                     'away_money':away_money[i+split_value]},index=[0]),ignore_index=True)
    svm_acc=round(accuracy_score(y_test,svm_pred)*100,1)
    boost_acc=round(accuracy_score(y_test,boost_pred)*100,1)
    combo_acc=round(accuracy_score(bets.real,bets.preds)*100,1)
    nat,equal=calc_return(bets)
    results_svm_xg=results_svm_xg.append(pd.DataFrame({'fold':j+1,'svm_acc':svm_acc,
                                                       'boost_acc':boost_acc,'combo_acc':combo_acc,
                                                       'nat':nat,'equal':equal},index=[0]),ignore_index=True)
print('Average SVM Accuracy Score: ',round(results_svm_xg.svm_acc.mean(),2))
print('Average XG Boost Accuracy Score: ',round(results_svm_xg.boost_acc.mean(),2))
print('Average Combined Accuracy Score: ',round(results_svm_xg.combo_acc.mean(),2))
print('Average Natural Wager Return on Risk: ',round(results_svm_xg.nat.mean(),2))
print('Average Equal Wager Return on Risk: ',round(results_svm_xg.equal.mean(),2))

Above is a re-creation of the winning model from the previous notebook.  Below I import the baseline results from the notebook Data Exploration and Visualization to compare the model to.  I use Kernel Density Estimation plots to visualize the distributions of the two datasets being compared.  I use Welch's t-test to try to confirm the null hypothesis in each case:

#### Accuracy score hypothesis:
$H_{0}$= The average accuracy score is the same for the baseline and for my model. 

$H_{a}$= The average accuracy score is the not same for the baseline and for my model. 
#### Natural Wager Return on Risk hypothesis:
$H_{0}$= The average return on risk is the same for the baseline and for my model. 

$H_{a}$= The average return on risk is the not same for the baseline and for my model. 
#### Equal Wager Return on Risk hypothesis:
$H_{0}$= The average return on risk is the same for the baseline and for my model. 

$H_{a}$= The average return on risk is the not same for the baseline and for my model. 

In [ ]:
pickle_in=open("baseline_accuracy.pickle","rb")
baseline_accuracy=pickle.load(pickle_in)
baseline_accuracy=pd.DataFrame(baseline_accuracy,columns=['acc'])
pickle_in=open("baseline_natural_ror.pickle","rb")
baseline_natural_ror=pickle.load(pickle_in)
baseline_natural_ror=pd.DataFrame(baseline_natural_ror,columns=['nat'])
pickle_in=open("baseline_equal_ror.pickle","rb")
baseline_equal_ror=pickle.load(pickle_in)
baseline_equal_ror=pd.DataFrame(baseline_equal_ror,columns=['equal'])


plt.figure(figsize=(10, 8))
baseline_accuracy.acc.plot.kde(label='Vegas')
results_svm_xg.combo_acc.plot.kde(label='Model')
plt.title('Accuracy Score: Vegas vs. Model')
plt.xlabel('Percent of Games Predicted Correctly')
plt.legend()
plt.show()
results = stats.ttest_ind(results_svm_xg.combo_acc,baseline_accuracy.acc, equal_var=False)
print('Coefficient: ',round(results[0],3))
print('p-value: ',round(results[1],3))

plt.figure(figsize=(10, 8))
baseline_natural_ror.nat.plot.kde(label='Vegas')
results_svm_xg.nat.plot.kde(label='Model')
plt.title('Natural Wager Return on Risk: Vegas vs. Model')
plt.xlabel('Percent Return on Risk')
plt.legend()
plt.show()
results = stats.ttest_ind(results_svm_xg.nat,baseline_natural_ror.nat, equal_var=False)
print('Coefficient: ',round(results[0],3))
print('p-value: ',round(results[1],3))

plt.figure(figsize=(10, 8))
baseline_equal_ror.equal.plot.kde(label='Vegas')
results_svm_xg.equal.plot.kde(label='Model')
plt.title('Equal Wager Return on Risk: Vegas vs. Model')
plt.xlabel('Percent Return on Risk')
plt.legend()
plt.show()
results = stats.ttest_ind(results_svm_xg.equal,baseline_equal_ror.equal, equal_var=False)
print('Coefficient: ',round(results[0],3))
print('p-value: ',round(results[1],3))

In all three cases the p-value is less than .05 and thus we can reject the null hypothesis in all three cases.  This test shows that my model predicts MLB games better than the Vegas odds in a statistically significant way.  

In order to get a sense of the potential usefulness of this model for making a profit in the betting markets, we can assume that the returns used in the above analysis are along the same lines as those that you would get from daily bets on games that the model predicts.  Obviously everyday won't be a clean 6% return; there will be a lot of variation and thus we can use these random sample return on risk as a proxy for daily return on risk.  Below is a visualization of a hypothetical betting account value over time using the four different methods that "re-invests" all profits.  Keep in mind, this is not an accurate representation of expected returns, it is simply a visualization of the difference in predictive power between my model and the Vegas odds.

In [ ]:
start=1000
vegas_nat=start
vegas_equal=start
model_nat=start
model_equal=start
perf=pd.DataFrame([])
for i in range(len(baseline_natural_ror.nat)):
    vegas_nat+=vegas_nat*(baseline_natural_ror.nat[i]*.01)
    vegas_equal+=start*(baseline_equal_ror.equal[i]*.01)
    model_nat+=start*(results_svm_xg.nat[i]*.01)
    model_equal+=start*(results_svm_xg.equal[i]*.01)
    perf=perf.append(pd.DataFrame({'day':i+1,'vegas_nat':round(vegas_nat,0),'vegas_equal':vegas_equal,
                                  'model_nat':model_nat,'model_equal':model_equal},index=[0]),
                     ignore_index=True)
plt.figure(figsize=(16,8))
plt.plot(perf.day,perf.model_equal,label="Model Equal       Final Value: ${0}".format(int(perf.model_equal[49])))
plt.plot(perf.day,perf.model_nat,label="Model Natural    Final Value: ${0}".format(int(perf.model_nat[49])))
plt.plot(perf.day,perf.vegas_nat,label="Vegas Natural    Final Value: ${0}".format(int(perf.vegas_nat[49]))) 
plt.plot(perf.day,perf.vegas_equal,label="Vegas Equal       Final Value: ${0}".format(int(perf.vegas_equal[49])))
plt.legend()   
plt.title('Hypothetical Betting Account Value Over Time')
plt.xlabel('Betting Day')
plt.ylabel('Betting Account Value')

# Conclusions:
- I was able to create a model that predicts MLB games more accurately and more profitably than the Vegas odds in a statistically significant way.  I did this by querying data from several online baseball databases and then optimizing several different classification models, before combining them to vote on the outcome of each game. 
- Oddly enough, it seems that always betting with the Vegas odds is a profitable strategy but using the model created in this project is potentially almose twice as profitable.

# Future Work:
For further exploration, I would use more types of data (new and highly advanced statistics) and more games from previous seasons. I would also automate the process of gathering the necessary data for today's games and publishing a report of which games to bet on.